In [1]:
import tensorflow as tf 
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import pandas
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Dropout, Activation
import pickle
import operator
from keras.models import Sequential

### Load data and model
import os
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf
import pickle
import datetime

from keras.models import load_model
from keras.utils import to_categorical
import sklearn.ensemble
import sklearn.preprocessing 
import lime 
import lime.lime_tabular
import re
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
dataset = 'twitter'

directory = "data/twitter_data_new/"
model_dir = directory + "model/"
model_name = 'model_twitter.h5'

# Region 1.2: Load previously created train and test data that is shuffled and scaled to be between 0 and 1

In [3]:
# Config
isModelTrained = True

In [76]:
x_train = np.load(directory + '/x_train.npy')
x_test = np.load(directory + '/x_test.npy')
y_train = np.load(directory + '/y_train.npy')
y_test = np.load(directory + '/y_test.npy')

In [4]:
train = x_train #modified_shruthi

num_features = x_train.shape[1]

if isModelTrained == False:
    # build DNNs
    DNNmodel = Sequential()
    DNNmodel.add(Dense(20,  input_dim = num_features, kernel_initializer="uniform",activation="relu"))
    DNNmodel.add(Dropout(0.5))
    DNNmodel.add(Dense(20, kernel_initializer="uniform", activation="relu"))
    DNNmodel.add(Dropout(0.5))
    DNNmodel.add(Dense(2))
    DNNmodel.add(Activation("softmax"))

    #compile Model
    DNNmodel.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # 3-fold validations
    history=DNNmodel.fit(x_train, y_train,validation_split=0.20,
              epochs=100, batch_size=256)
    
    
    # Save the model
    DNNmodel.save(model_dir + model_name)

else:
    
    # Load the model (Starting point)
    DNNmodel = load_model(model_dir + model_name)

    W = DNNmodel.get_weights()
    W={'weights1': W[0], 
       'weights2': W[2],
       'weights3': W[4],
       'biases1' : W[1],
       'biases2' : W[3],    
       'biases3' : W[5]    
      }
    
#evaluation
score = DNNmodel.evaluate(x_test, y_test, batch_size=256)
print("Test Accuracy",score)
print("\n%s: %.2f%%" % (DNNmodel.metrics_names[1], score[1]*100))

# evaluation through predict function

y_pred = DNNmodel.predict(x_test)

tp =0
fp =0
tn =0
fn =0
for i in range(len(y_pred)):
    pred_class = np.argmax(y_pred[i])
    true_class = np.argmax(y_test[i])
    if(pred_class == true_class):
        if true_class == 1:
            tp += 1
        else:
            tn += 1
    else:
        if true_class == 1:
            fn += 1
        else:
            fp += 1        
    
precision = tp*100/ (tp+fp)
recall = tp*100/ (tp+fn)
acc = (tp+tn)*100/(tp+tn+fp+fn)

total_actual_positives = tp+fn
total_actual_negatives = fp+tn

print("Accuracy is {}%".format(acc))
print('tp: {} fp: {} tn: {} fn:{} '.format(tp,fp,tn,fn))
print("precision: {}%   recall: {}%".format(precision, recall))

print("Total true +ves: {} Total true -ves: {}".format(total_actual_positives, total_actual_negatives))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
8300/8300 [==============================] - 0s 11us/step
Test Accuracy [0.16959909632263415, 0.9380722641944885]

accuracy: 93.81%
Accuracy is 93.80722891566265%
tp: 4320 fp: 387 tn: 3466 fn:127 
precision: 91.77820267686424%   recall: 97.14414211828199%
Total true +ves: 4447 Total true -ves: 3853


In [145]:
num_ben=0
num_mal=0
print("TRAIN DATA")
print("Number of samples ", len(x_train))
for i in range(len(x_train)):
    if np.argmax(y_train[i])==0:
        num_ben+=1
    else:
        num_mal+=1
print("Number of benign samples ", num_ben)   
print("Number of malignant samples ", num_mal)

num_ben=0
num_mal=0
print("TEST DATA")
print("Number of samples ", len(x_test))
for i in range(len(x_test)):
    if np.argmax(y_test[i])==0:
        num_ben+=1
    else:
        num_mal+=1
print("Number of benign samples ", num_ben)   
print("Number of malignant samples ", num_mal)

TRAIN DATA
Number of samples  33199
Number of benign samples  15423
Number of malignant samples  17776
TEST DATA
Number of samples  8300
Number of benign samples  3853
Number of malignant samples  4447


In [5]:
tws_feature_names = ['age', 'NumerOfFollowings', 'NumberOfFollowers', 'NumberOfTweets', 'LengthOfScreenName', 
                         'LengthOfDescriptionInUserProfile', 'num_tws', 'ratio_question', 'ratio_exclam', 
                     'len_tws', 'speed_tws', 'num_url', 'ratio_url', 'num_at', 'ratio_at', 'num_RT', 'ratio_RT',
                    'num_uniq_at', 'ratio_uniq_at', 'num_reply', 'ratio_reply', 'num_hash', 'ratio_hash', 'jacc_tw',
                    'jacc_url', 'compress_ratio', 'spam word ratio', 'FollowingChangeRatio']
tws_target_names = ['benign', 'malicious']

In [6]:
# y_test_class may not be all same as y_pred_class due to test error
y_pred_class = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)
ind_mal = np.where(y_test_class == 1)[0]
ind_ben = np.where(y_test_class == 0)[0]
print('ben: ',ind_ben[0:10])
print('mal: ',ind_mal[0:10])

ben:  [ 0  1  3  6  7 10 11 12 13 14]
mal:  [ 2  4  5  8  9 15 16 18 19 21]


# Evaluating the model on clean samples

In [ ]:
samples = x_train #modified_shruthi
    
pred_x = prediction(samples)
benign_indices = np.where(pred_x==0)[0]

# keep only benign samples
samples = samples[benign_indices]

# Generate interpretations as defender for clean examples

In [64]:
# Which experiments to run?
gen_intr_for_def = True
gen_intr_for_adv = False
train_intr_model = True
test_intr_model = False
eval_intr_model_on_adv = True

In [12]:
predict_fn = lambda x: DNNmodel.predict(x)
#prediction = lambda x: np.argmax(DNNmodel.predict(x.reshape(-1, num_features)), axis=1)

def prediction(data):
    pred = DNNmodel.predict(data)
    pred_class=np.zeros(len(data))
    for i in range(len(data)):
        pred_class[i] = np.argmax(pred[i])
    return pred_class

In [44]:
import re
#generate interpretations for the benign and adverserial smaples (ones that have been classified as 0 by F())
def generate_interpretation(train, samples, role, clean_or_adv):

    # this is the desired matrix we want to make from samples
    intrs = np.empty(samples.shape)

    # we will consider top-k features
    k = 5

    #Create the explainer
    np.random.seed(1)

    """
    Function is defined in https://github.com/marcotcr/lime/blob/master/lime/lime_tabular.py
    """
    explainer = lime.lime_tabular.LimeTabularExplainer(train,
                                                       class_names=tws_target_names, 
                                                       feature_names = tws_feature_names)
                                                       #categorical_features=categorical_features, 
                                                       #categorical_names=categorical_names, 
                                                       #kernel_width=3, 
                                                       #verbose=False)
    num_samples = samples.shape[0]

    for i in range(num_samples):
        print('Running for {}/{}'.format(i,num_samples))
        sample = samples[i]

        intr_sample = np.copy(sample)
        intr_sample[:] = 0 #modified_shruthi

        # This line takes the most time as it produces explaination
        #produces explanation for sample i
        exp = explainer.explain_instance(sample, predict_fn, num_features=5)
        
        exp_l = exp.as_list(label=1) 

        p = predict_fn(sample.reshape(1,-1))[0,0]

        for j,exp_feature in enumerate(exp_l):

            if j>=k:
                break
            tmp = exp_feature[0].split(" ")
            res=''
            for t in tmp:
                if t[0].isalpha() and not res:
                    res+=t
                elif t[0].isalpha() and res:
                    res=res+" "+t
            #print(exp_feature[0],res)
            
            feature_index = tws_feature_names.index(res)

            # we only retain important features
            intr_sample[feature_index] = sample[feature_index]

        intrs[i] = intr_sample

    out_directory = 'data/tw_mal_ssk/intr/'+ role + '/' + clean_or_adv
    if not os.path.exists(out_directory):
            os.makedirs(out_directory)

    np.save(out_directory+'/tw_mal_ssk_intrs.npy', intrs)
    return intrs

In [45]:
if gen_intr_for_def:
    # generate interpretations as defender for clean benign examples
    role='defender'
    samples = x_train #modified_shruthi
    
    pred_x = prediction(samples)
    benign_indices = np.where(pred_x==0)[0]

    # keep only benign samples
    samples = samples[benign_indices]
    #producing interpretations for clean benign samples
    intrs_clean_benign = generate_interpretation(train, samples, role, 'clean')

    # Generate interpretations for adversarial examples which originated from malware samples of training set
    ##directory = 'data/pdf_mal/'+role+'/target_next/jsma/'
    ##x_adv = np.load(directory+'/x_adv.npy')
    #producing interpretations for adverserial samples
    ##intrs_adv = generate_interpretation(train, x_adv, role, 'adv')

Running for 0/14478
Running for 1/14478
Running for 2/14478
Running for 3/14478
Running for 4/14478
Running for 5/14478
Running for 6/14478
Running for 7/14478
Running for 8/14478
Running for 9/14478
Running for 10/14478
Running for 11/14478
Running for 12/14478
Running for 13/14478
Running for 14/14478
Running for 15/14478
Running for 16/14478
Running for 17/14478
Running for 18/14478
Running for 19/14478
Running for 20/14478
Running for 21/14478
Running for 22/14478
Running for 23/14478
Running for 24/14478
Running for 25/14478
Running for 26/14478
Running for 27/14478
Running for 28/14478
Running for 29/14478
Running for 30/14478
Running for 31/14478
Running for 32/14478
Running for 33/14478
Running for 34/14478
Running for 35/14478
Running for 36/14478
Running for 37/14478
Running for 38/14478
Running for 39/14478
Running for 40/14478
Running for 41/14478
Running for 42/14478
Running for 43/14478
Running for 44/14478
Running for 45/14478
Running for 46/14478
Running for 47/14478
Ru

Running for 378/14478
Running for 379/14478
Running for 380/14478
Running for 381/14478
Running for 382/14478
Running for 383/14478
Running for 384/14478
Running for 385/14478
Running for 386/14478
Running for 387/14478
Running for 388/14478
Running for 389/14478
Running for 390/14478
Running for 391/14478
Running for 392/14478
Running for 393/14478
Running for 394/14478
Running for 395/14478
Running for 396/14478
Running for 397/14478
Running for 398/14478
Running for 399/14478
Running for 400/14478
Running for 401/14478
Running for 402/14478
Running for 403/14478
Running for 404/14478
Running for 405/14478
Running for 406/14478
Running for 407/14478
Running for 408/14478
Running for 409/14478
Running for 410/14478
Running for 411/14478
Running for 412/14478
Running for 413/14478
Running for 414/14478
Running for 415/14478
Running for 416/14478
Running for 417/14478
Running for 418/14478
Running for 419/14478
Running for 420/14478
Running for 421/14478
Running for 422/14478
Running fo

Running for 751/14478
Running for 752/14478
Running for 753/14478
Running for 754/14478
Running for 755/14478
Running for 756/14478
Running for 757/14478
Running for 758/14478
Running for 759/14478
Running for 760/14478
Running for 761/14478
Running for 762/14478
Running for 763/14478
Running for 764/14478
Running for 765/14478
Running for 766/14478
Running for 767/14478
Running for 768/14478
Running for 769/14478
Running for 770/14478
Running for 771/14478
Running for 772/14478
Running for 773/14478
Running for 774/14478
Running for 775/14478
Running for 776/14478
Running for 777/14478
Running for 778/14478
Running for 779/14478
Running for 780/14478
Running for 781/14478
Running for 782/14478
Running for 783/14478
Running for 784/14478
Running for 785/14478
Running for 786/14478
Running for 787/14478
Running for 788/14478
Running for 789/14478
Running for 790/14478
Running for 791/14478
Running for 792/14478
Running for 793/14478
Running for 794/14478
Running for 795/14478
Running fo

Running for 1119/14478
Running for 1120/14478
Running for 1121/14478
Running for 1122/14478
Running for 1123/14478
Running for 1124/14478
Running for 1125/14478
Running for 1126/14478
Running for 1127/14478
Running for 1128/14478
Running for 1129/14478
Running for 1130/14478
Running for 1131/14478
Running for 1132/14478
Running for 1133/14478
Running for 1134/14478
Running for 1135/14478
Running for 1136/14478
Running for 1137/14478
Running for 1138/14478
Running for 1139/14478
Running for 1140/14478
Running for 1141/14478
Running for 1142/14478
Running for 1143/14478
Running for 1144/14478
Running for 1145/14478
Running for 1146/14478
Running for 1147/14478
Running for 1148/14478
Running for 1149/14478
Running for 1150/14478
Running for 1151/14478
Running for 1152/14478
Running for 1153/14478
Running for 1154/14478
Running for 1155/14478
Running for 1156/14478
Running for 1157/14478
Running for 1158/14478
Running for 1159/14478
Running for 1160/14478
Running for 1161/14478
Running for

Running for 1476/14478
Running for 1477/14478
Running for 1478/14478
Running for 1479/14478
Running for 1480/14478
Running for 1481/14478
Running for 1482/14478
Running for 1483/14478
Running for 1484/14478
Running for 1485/14478
Running for 1486/14478
Running for 1487/14478
Running for 1488/14478
Running for 1489/14478
Running for 1490/14478
Running for 1491/14478
Running for 1492/14478
Running for 1493/14478
Running for 1494/14478
Running for 1495/14478
Running for 1496/14478
Running for 1497/14478
Running for 1498/14478
Running for 1499/14478
Running for 1500/14478
Running for 1501/14478
Running for 1502/14478
Running for 1503/14478
Running for 1504/14478
Running for 1505/14478
Running for 1506/14478
Running for 1507/14478
Running for 1508/14478
Running for 1509/14478
Running for 1510/14478
Running for 1511/14478
Running for 1512/14478
Running for 1513/14478
Running for 1514/14478
Running for 1515/14478
Running for 1516/14478
Running for 1517/14478
Running for 1518/14478
Running for

...

Running for 12778/14478
Running for 12779/14478
Running for 12780/14478
Running for 12781/14478
Running for 12782/14478
Running for 12783/14478
Running for 12784/14478
Running for 12785/14478
Running for 12786/14478
Running for 12787/14478
Running for 12788/14478
Running for 12789/14478
Running for 12790/14478
Running for 12791/14478
Running for 12792/14478
Running for 12793/14478
Running for 12794/14478
Running for 12795/14478
Running for 12796/14478
Running for 12797/14478
Running for 12798/14478
Running for 12799/14478
Running for 12800/14478
Running for 12801/14478
Running for 12802/14478
Running for 12803/14478
Running for 12804/14478
Running for 12805/14478
Running for 12806/14478
Running for 12807/14478
Running for 12808/14478
Running for 12809/14478
Running for 12810/14478
Running for 12811/14478
Running for 12812/14478
Running for 12813/14478
Running for 12814/14478
Running for 12815/14478
Running for 12816/14478
Running for 12817/14478
Running for 12818/14478
Running for 1281

Running for 13120/14478
Running for 13121/14478
Running for 13122/14478
Running for 13123/14478
Running for 13124/14478
Running for 13125/14478
Running for 13126/14478
Running for 13127/14478
Running for 13128/14478
Running for 13129/14478
Running for 13130/14478
Running for 13131/14478
Running for 13132/14478
Running for 13133/14478
Running for 13134/14478
Running for 13135/14478
Running for 13136/14478
Running for 13137/14478
Running for 13138/14478
Running for 13139/14478
Running for 13140/14478
Running for 13141/14478
Running for 13142/14478
Running for 13143/14478
Running for 13144/14478
Running for 13145/14478
Running for 13146/14478
Running for 13147/14478
Running for 13148/14478
Running for 13149/14478
Running for 13150/14478
Running for 13151/14478
Running for 13152/14478
Running for 13153/14478
Running for 13154/14478
Running for 13155/14478
Running for 13156/14478
Running for 13157/14478
Running for 13158/14478
Running for 13159/14478
Running for 13160/14478
Running for 1316

Running for 13462/14478
Running for 13463/14478
Running for 13464/14478
Running for 13465/14478
Running for 13466/14478
Running for 13467/14478
Running for 13468/14478
Running for 13469/14478
Running for 13470/14478
Running for 13471/14478
Running for 13472/14478
Running for 13473/14478
Running for 13474/14478
Running for 13475/14478
Running for 13476/14478
Running for 13477/14478
Running for 13478/14478
Running for 13479/14478
Running for 13480/14478
Running for 13481/14478
Running for 13482/14478
Running for 13483/14478
Running for 13484/14478
Running for 13485/14478
Running for 13486/14478
Running for 13487/14478
Running for 13488/14478
Running for 13489/14478
Running for 13490/14478
Running for 13491/14478
Running for 13492/14478
Running for 13493/14478
Running for 13494/14478
Running for 13495/14478
Running for 13496/14478
Running for 13497/14478
Running for 13498/14478
Running for 13499/14478
Running for 13500/14478
Running for 13501/14478
Running for 13502/14478
Running for 1350

Running for 13804/14478
Running for 13805/14478
Running for 13806/14478
Running for 13807/14478
Running for 13808/14478
Running for 13809/14478
Running for 13810/14478
Running for 13811/14478
Running for 13812/14478
Running for 13813/14478
Running for 13814/14478
Running for 13815/14478
Running for 13816/14478
Running for 13817/14478
Running for 13818/14478
Running for 13819/14478
Running for 13820/14478
Running for 13821/14478
Running for 13822/14478
Running for 13823/14478
Running for 13824/14478
Running for 13825/14478
Running for 13826/14478
Running for 13827/14478
Running for 13828/14478
Running for 13829/14478
Running for 13830/14478
Running for 13831/14478
Running for 13832/14478
Running for 13833/14478
Running for 13834/14478
Running for 13835/14478
Running for 13836/14478
Running for 13837/14478
Running for 13838/14478
Running for 13839/14478
Running for 13840/14478
Running for 13841/14478
Running for 13842/14478
Running for 13843/14478
Running for 13844/14478
Running for 1384

Running for 14146/14478
Running for 14147/14478
Running for 14148/14478
Running for 14149/14478
Running for 14150/14478
Running for 14151/14478
Running for 14152/14478
Running for 14153/14478
Running for 14154/14478
Running for 14155/14478
Running for 14156/14478
Running for 14157/14478
Running for 14158/14478
Running for 14159/14478
Running for 14160/14478
Running for 14161/14478
Running for 14162/14478
Running for 14163/14478
Running for 14164/14478
Running for 14165/14478
Running for 14166/14478
Running for 14167/14478
Running for 14168/14478
Running for 14169/14478
Running for 14170/14478
Running for 14171/14478
Running for 14172/14478
Running for 14173/14478
Running for 14174/14478
Running for 14175/14478
Running for 14176/14478
Running for 14177/14478
Running for 14178/14478
Running for 14179/14478
Running for 14180/14478
Running for 14181/14478
Running for 14182/14478
Running for 14183/14478
Running for 14184/14478
Running for 14185/14478
Running for 14186/14478
Running for 1418

In [140]:
print("Number of samples that are predicted as benign ", benign_indices.shape)

Number of samples that are predicted as benign  (14478,)


In [57]:
#producing interpretations for adverserial samples samples
attacked_samples_y = np.load(directory + '/'  + role + '/twitter/target_next/jsma/y_class_adv.npy' )
adv_indices = np.where(attacked_samples_y==0)[0]
attacked_samples_x = np.load(directory + '/'  + role + '/twitter/target_next/jsma/xadvclev_twitter.npy')
adv_x=attacked_samples_x[adv_indices]    

intrs_adv = generate_interpretation(train, adv_x, role, 'adv')

Running for 0/12713
Running for 1/12713
Running for 2/12713
Running for 3/12713
Running for 4/12713
Running for 5/12713
Running for 6/12713
Running for 7/12713
Running for 8/12713
Running for 9/12713
Running for 10/12713
Running for 11/12713
Running for 12/12713
Running for 13/12713
Running for 14/12713
Running for 15/12713
Running for 16/12713
Running for 17/12713
Running for 18/12713
Running for 19/12713
Running for 20/12713
Running for 21/12713
Running for 22/12713
Running for 23/12713
Running for 24/12713
Running for 25/12713
Running for 26/12713
Running for 27/12713
Running for 28/12713
Running for 29/12713
Running for 30/12713
Running for 31/12713
Running for 32/12713
Running for 33/12713
Running for 34/12713
Running for 35/12713
Running for 36/12713
Running for 37/12713
Running for 38/12713
Running for 39/12713
Running for 40/12713
Running for 41/12713
Running for 42/12713
Running for 43/12713
Running for 44/12713
Running for 45/12713
Running for 46/12713
Running for 47/12713
Ru

Running for 378/12713
Running for 379/12713
Running for 380/12713
Running for 381/12713
Running for 382/12713
Running for 383/12713
Running for 384/12713
Running for 385/12713
Running for 386/12713
Running for 387/12713
Running for 388/12713
Running for 389/12713
Running for 390/12713
Running for 391/12713
Running for 392/12713
Running for 393/12713
Running for 394/12713
Running for 395/12713
Running for 396/12713
Running for 397/12713
Running for 398/12713
Running for 399/12713
Running for 400/12713
Running for 401/12713
Running for 402/12713
Running for 403/12713
Running for 404/12713
Running for 405/12713
Running for 406/12713
Running for 407/12713
Running for 408/12713
Running for 409/12713
Running for 410/12713
Running for 411/12713
Running for 412/12713
Running for 413/12713
Running for 414/12713
Running for 415/12713
Running for 416/12713
Running for 417/12713
Running for 418/12713
Running for 419/12713
Running for 420/12713
Running for 421/12713
Running for 422/12713
Running fo

Running for 751/12713
Running for 752/12713
Running for 753/12713
Running for 754/12713
Running for 755/12713
Running for 756/12713
Running for 757/12713
Running for 758/12713
Running for 759/12713
Running for 760/12713
Running for 761/12713
Running for 762/12713
Running for 763/12713
Running for 764/12713
Running for 765/12713
Running for 766/12713
Running for 767/12713
Running for 768/12713
Running for 769/12713
Running for 770/12713
Running for 771/12713
Running for 772/12713
Running for 773/12713
Running for 774/12713
Running for 775/12713
Running for 776/12713
Running for 777/12713
Running for 778/12713
Running for 779/12713
Running for 780/12713
Running for 781/12713
Running for 782/12713
Running for 783/12713
Running for 784/12713
Running for 785/12713
Running for 786/12713
Running for 787/12713
Running for 788/12713
Running for 789/12713
Running for 790/12713
Running for 791/12713
Running for 792/12713
Running for 793/12713
Running for 794/12713
Running for 795/12713
Running fo

Running for 1119/12713
Running for 1120/12713
Running for 1121/12713
Running for 1122/12713
Running for 1123/12713
Running for 1124/12713
Running for 1125/12713
Running for 1126/12713
Running for 1127/12713
Running for 1128/12713
Running for 1129/12713
Running for 1130/12713
Running for 1131/12713
Running for 1132/12713
Running for 1133/12713
Running for 1134/12713
Running for 1135/12713
Running for 1136/12713
Running for 1137/12713
Running for 1138/12713
Running for 1139/12713
Running for 1140/12713
Running for 1141/12713
Running for 1142/12713
Running for 1143/12713
Running for 1144/12713
Running for 1145/12713
Running for 1146/12713
Running for 1147/12713
Running for 1148/12713
Running for 1149/12713
Running for 1150/12713
Running for 1151/12713
Running for 1152/12713
Running for 1153/12713
Running for 1154/12713
Running for 1155/12713
Running for 1156/12713
Running for 1157/12713
Running for 1158/12713
Running for 1159/12713
Running for 1160/12713
Running for 1161/12713
Running for

Running for 1476/12713
Running for 1477/12713
Running for 1478/12713
Running for 1479/12713
Running for 1480/12713
Running for 1481/12713
Running for 1482/12713
Running for 1483/12713
Running for 1484/12713
Running for 1485/12713
Running for 1486/12713
Running for 1487/12713
Running for 1488/12713
Running for 1489/12713
Running for 1490/12713
Running for 1491/12713
Running for 1492/12713
Running for 1493/12713
Running for 1494/12713
Running for 1495/12713
Running for 1496/12713
Running for 1497/12713
Running for 1498/12713
Running for 1499/12713
Running for 1500/12713
Running for 1501/12713
Running for 1502/12713
Running for 1503/12713
Running for 1504/12713
Running for 1505/12713
Running for 1506/12713
Running for 1507/12713
Running for 1508/12713
Running for 1509/12713
Running for 1510/12713
Running for 1511/12713
Running for 1512/12713
Running for 1513/12713
Running for 1514/12713
Running for 1515/12713
Running for 1516/12713
Running for 1517/12713
Running for 1518/12713
Running for

...

Running for 11068/12713
Running for 11069/12713
Running for 11070/12713
Running for 11071/12713
Running for 11072/12713
Running for 11073/12713
Running for 11074/12713
Running for 11075/12713
Running for 11076/12713
Running for 11077/12713
Running for 11078/12713
Running for 11079/12713
Running for 11080/12713
Running for 11081/12713
Running for 11082/12713
Running for 11083/12713
Running for 11084/12713
Running for 11085/12713
Running for 11086/12713
Running for 11087/12713
Running for 11088/12713
Running for 11089/12713
Running for 11090/12713
Running for 11091/12713
Running for 11092/12713
Running for 11093/12713
Running for 11094/12713
Running for 11095/12713
Running for 11096/12713
Running for 11097/12713
Running for 11098/12713
Running for 11099/12713
Running for 11100/12713
Running for 11101/12713
Running for 11102/12713
Running for 11103/12713
Running for 11104/12713
Running for 11105/12713
Running for 11106/12713
Running for 11107/12713
Running for 11108/12713
Running for 1110

Running for 11410/12713
Running for 11411/12713
Running for 11412/12713
Running for 11413/12713
Running for 11414/12713
Running for 11415/12713
Running for 11416/12713
Running for 11417/12713
Running for 11418/12713
Running for 11419/12713
Running for 11420/12713
Running for 11421/12713
Running for 11422/12713
Running for 11423/12713
Running for 11424/12713
Running for 11425/12713
Running for 11426/12713
Running for 11427/12713
Running for 11428/12713
Running for 11429/12713
Running for 11430/12713
Running for 11431/12713
Running for 11432/12713
Running for 11433/12713
Running for 11434/12713
Running for 11435/12713
Running for 11436/12713
Running for 11437/12713
Running for 11438/12713
Running for 11439/12713
Running for 11440/12713
Running for 11441/12713
Running for 11442/12713
Running for 11443/12713
Running for 11444/12713
Running for 11445/12713
Running for 11446/12713
Running for 11447/12713
Running for 11448/12713
Running for 11449/12713
Running for 11450/12713
Running for 1145

Running for 11752/12713
Running for 11753/12713
Running for 11754/12713
Running for 11755/12713
Running for 11756/12713
Running for 11757/12713
Running for 11758/12713
Running for 11759/12713
Running for 11760/12713
Running for 11761/12713
Running for 11762/12713
Running for 11763/12713
Running for 11764/12713
Running for 11765/12713
Running for 11766/12713
Running for 11767/12713
Running for 11768/12713
Running for 11769/12713
Running for 11770/12713
Running for 11771/12713
Running for 11772/12713
Running for 11773/12713
Running for 11774/12713
Running for 11775/12713
Running for 11776/12713
Running for 11777/12713
Running for 11778/12713
Running for 11779/12713
Running for 11780/12713
Running for 11781/12713
Running for 11782/12713
Running for 11783/12713
Running for 11784/12713
Running for 11785/12713
Running for 11786/12713
Running for 11787/12713
Running for 11788/12713
Running for 11789/12713
Running for 11790/12713
Running for 11791/12713
Running for 11792/12713
Running for 1179

Running for 12094/12713
Running for 12095/12713
Running for 12096/12713
Running for 12097/12713
Running for 12098/12713
Running for 12099/12713
Running for 12100/12713
Running for 12101/12713
Running for 12102/12713
Running for 12103/12713
Running for 12104/12713
Running for 12105/12713
Running for 12106/12713
Running for 12107/12713
Running for 12108/12713
Running for 12109/12713
Running for 12110/12713
Running for 12111/12713
Running for 12112/12713
Running for 12113/12713
Running for 12114/12713
Running for 12115/12713
Running for 12116/12713
Running for 12117/12713
Running for 12118/12713
Running for 12119/12713
Running for 12120/12713
Running for 12121/12713
Running for 12122/12713
Running for 12123/12713
Running for 12124/12713
Running for 12125/12713
Running for 12126/12713
Running for 12127/12713
Running for 12128/12713
Running for 12129/12713
Running for 12130/12713
Running for 12131/12713
Running for 12132/12713
Running for 12133/12713
Running for 12134/12713
Running for 1213

Running for 12436/12713
Running for 12437/12713
Running for 12438/12713
Running for 12439/12713
Running for 12440/12713
Running for 12441/12713
Running for 12442/12713
Running for 12443/12713
Running for 12444/12713
Running for 12445/12713
Running for 12446/12713
Running for 12447/12713
Running for 12448/12713
Running for 12449/12713
Running for 12450/12713
Running for 12451/12713
Running for 12452/12713
Running for 12453/12713
Running for 12454/12713
Running for 12455/12713
Running for 12456/12713
Running for 12457/12713
Running for 12458/12713
Running for 12459/12713
Running for 12460/12713
Running for 12461/12713
Running for 12462/12713
Running for 12463/12713
Running for 12464/12713
Running for 12465/12713
Running for 12466/12713
Running for 12467/12713
Running for 12468/12713
Running for 12469/12713
Running for 12470/12713
Running for 12471/12713
Running for 12472/12713
Running for 12473/12713
Running for 12474/12713
Running for 12475/12713
Running for 12476/12713
Running for 1247

In [189]:
# pick a random index from benign_test_indices

q = np.random.randint(0, len(adv_x))

"""
explain_instance function is defined in https://github.com/marcotcr/lime/blob/master/lime/lime_tabular.py
"""

exp = explainer.explain_instance(adv_x[q], predict_fn, num_features=5)
#way of interpreting - fancy
exp.show_in_notebook()

In [81]:
from sklearn.model_selection import train_test_split
# Train the interpretation based model
intrs_clean_benign_y = np.zeros(intrs_clean_benign.shape[0])
intrs_clean_benign_x = intrs_clean_benign
intrs_adv_y = np.ones(intrs_adv.shape[0])
intrs_adv_x = intrs_adv

intr_x = np.vstack((intrs_clean_benign_x, intrs_adv_x))
intr_y = np.hstack((intrs_clean_benign_y,intrs_adv_y))

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(intr_x, intr_y, test_size=0.33, random_state=42)

categorical_features = range(intrs_clean_benign_x.shape[1])

#Set label into one hot encoder
encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=categorical_features)
encoder.fit(intr_x)
encoded_train = encoder.transform(x_train_rf)

model_file = "data/tw_mal_ssk/model/rf_intr_model.pkl"

if train_intr_model:
    # Fit a model
    rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
    rf.fit(encoded_train, y_train_rf)
    
    # Save to file in the current working directory
    pkl_filename = "data/tw_mal_ssk/model/rf_intr_model.pkl"
    with open(pkl_filename, 'wb') as file:
        pickle.dump(rf, file)   
else:
    # Load from file
    with open(model_file, 'rb') as file:
        rf = pickle.load(file)

//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [83]:
y_test_class = np.argmax(y_test, axis=1)
print(y_test_class)

[0 0 1 ... 0 1 1]


In [150]:
num_ben=0
num_adv=0
print("Train data")
for i in range(encoded_train.shape[0]):
    if y_train_rf[i]==0:
        num_ben+=1
    else:
        num_adv+=1
print("Number of samples ", len(y_train_rf))        
print("Number of benign samples ", num_ben)
print("Number of adversarial samples ", num_adv)

Train data
Number of samples  18217
Number of benign samples  9690
Number of adversarial samples  8527


In [89]:
# Generate intr for adversarial example of adversary and classify its intr as normal or abnormal

#Create the explainer
np.random.seed(1)

"""
Function is defined in https://github.com/marcotcr/lime/blob/master/lime/lime_tabular.py
"""
explainer = lime.lime_tabular.LimeTabularExplainer(x_train,
                                                   class_names=['benign', 'malware'], 
                                                   feature_names = tws_feature_names)
                                                   #categorical_features=categorical_features, 
                                                   #categorical_names=categorical_names, 
                                                   #kernel_width=3, 
                                                   #verbose=False)

In [92]:
def preprocess_feature_vector(sample, exp_l):
    intr_sample = np.copy(sample)
    intr_sample[:] = 0

    for j, exp_feature in enumerate(exp_l):
        
        tmp = exp_feature[0].split(" ")
        res=''
        for t in tmp:
            if t[0].isalpha() and not res:
                res+=t
            elif t[0].isalpha() and res:
                res=res+" "+t
        #print(exp_feature[0],res)
            
        feature_index = tws_feature_names.index(res)

        # we only retain important features
        intr_sample[feature_index] = sample[feature_index]

    return intr_sample

In [94]:
intr_predict_fn = lambda x: rf.predict_proba(encoder.transform(x))
intr_predict_class = lambda x: np.argmax(rf.predict_proba(encoder.transform(x)), axis=1)

In [96]:
#running for test set : benign + adverserial
role='attacker'
out_directory = 'data/tw_mal_ssk/intr/'+ role + '/'

if eval_intr_model_on_adv:
    ## FOR BENIGN
    print('runing for benign')
    benign_test_indices = np.where(y_test_class==0)[0]
    x_test_b = x_test[benign_test_indices]
    y_pred_class_test = np.empty(x_test_b.shape[0])

    for i in range(x_test_b.shape[0]):
        if i%10 == 0:
            print('i: ',i)
        sample = x_test_b[i]

        exp = explainer.explain_instance(sample, predict_fn, num_features=5)
        exp_l = exp.as_list(label=1)
        intr_sample = preprocess_feature_vector(sample, exp_l)
        y_pred_class_test[i] = intr_predict_class(intr_sample.reshape(1,-1))[0]

     
    if not os.path.exists(out_directory):
            os.makedirs(out_directory)

    np.save(out_directory+'/y_pred_class_test.npy', y_pred_class_test)

    ### FOR ADV
    directory = 'data/twitter_data_new/adversary/twitter/target_next/jsma/'
    x_adv = np.load(directory+'/xadvclev_twitter.npy')
    y_pred_class_adv = np.empty(x_adv.shape[0])

    print('runing for adv')
    for i in range(x_adv.shape[0]):
        if i%100 == 0:
            print('i: ',i)
        sample = x_adv[i]
        exp = explainer.explain_instance(sample, predict_fn, num_features=5)
        exp_l = exp.as_list(label=1)
        intr_sample = preprocess_feature_vector(sample, exp_l)
        try:
            y_pred_class_adv[i] = intr_predict_class(intr_sample.reshape(1,-1))[0]
        except:
            print('failed for index,'i)
            y_pred_class_adv[i] = 0

    np.save(out_directory+'/y_pred_class_adv.npy', y_pred_class_adv)
    
else:
    print('loading classification results')
    y_pred_class_test = np.load(out_directory+'/y_pred_class_test.npy')
    y_pred_class_adv = np.load(out_directory+'/y_pred_class_adv.npy')

runing for benign
i:  0
i:  10
i:  20
i:  30
i:  40
i:  50
i:  60
i:  70
i:  80
i:  90
i:  100
i:  110
i:  120
i:  130
i:  140
i:  150
i:  160
i:  170
i:  180
i:  190
i:  200
i:  210
i:  220
i:  230
i:  240
i:  250
i:  260
i:  270
i:  280
i:  290
i:  300
i:  310
i:  320
i:  330
i:  340
i:  350
i:  360
i:  370
i:  380
i:  390
i:  400
i:  410
i:  420
i:  430
i:  440
i:  450
i:  460
i:  470
i:  480
i:  490
i:  500
i:  510
i:  520
i:  530
i:  540
i:  550
i:  560
i:  570
i:  580
i:  590
i:  600
i:  610
i:  620
i:  630
i:  640
i:  650
i:  660
i:  670
i:  680
i:  690
i:  700
i:  710
i:  720
i:  730
i:  740
i:  750
i:  760
i:  770
i:  780
i:  790
i:  800
i:  810
i:  820
i:  830
i:  840
i:  850
i:  860
i:  870
i:  880
i:  890
i:  900
i:  910
i:  920
i:  930
i:  940
i:  950
i:  960
i:  970
i:  980
i:  990
i:  1000
i:  1010
i:  1020
i:  1030
i:  1040
i:  1050
i:  1060
i:  1070
i:  1080
i:  1090
i:  1100
i:  1110
i:  1120
i:  1130
i:  1140
i:  1150
i:  1160
i:  1170
i:  1180
i:  1190
i:  1200
i:  

ValueError: unknown categorical feature present [1] during transform.

In [151]:
print("Test data")
num_ben= x_test_b.shape[0]
num_adv= x_adv.shape[0]
print("Number of samples ", num_ben+num_adv)        
print("Number of benign samples ", num_ben)
print("Number of adversarial samples ", num_adv)

Test data
Number of samples  8293
Number of benign samples  3853
Number of adversarial samples  4440


In [135]:
#calculation of false positive ratio and fasle negative ratio
pred_benign_test = np.load(out_directory+'/y_pred_class_test.npy')
false_positives=0
true_positives=0
for i in range(pred_benign_test.shape[0]):
    if pred_benign_test[i]==1:
        false_positives+=1
    elif pred_benign_test[i]==0:
        true_positives+=1
        
pred_adv_test = np.load(out_directory+'/y_pred_class_adv.npy')
#print(pred_adv_test)
false_negatives=0
true_negatives=0
for j in range(pred_adv_test.shape[0]):
    if pred_adv_test[i]==0:
        false_negatives+=1
    elif pred_adv_test[i]==1:
        true_negatives+=1
print("False Positive Rate is {}/{} {}".format(false_positives,pred_benign_test.shape[0], false_positives/pred_benign_test.shape[0]))  
print("False Negative Rate is {}/{} {}".format(false_negatives,pred_adv_test.shape[0], false_negatives/pred_adv_test.shape[0]))



False Positive Rate is 1538/3853 0.39916947832857513
False Negative Rate is 0/4440 0.0


In [136]:
print(true_positives)
print(false_negatives)
precision_test = true_positives/(true_positives+false_positives)
recall_test = true_positives/ (true_positives+false_negatives)

print("Precision ",precision_test)
print("Recall ", recall_test
     )

2315
0
Precision  0.6008305216714248
Recall  1.0


In [127]:
#calculation of accuracy
correct=0
for i in range(pred_benign_test.shape[0]):
    if pred_benign_test[i]==0:
        correct+=1

for j in range(pred_adv_test.shape[0]):
    if pred_adv_test[i]==1:
        correct+=1
        
print("Accuracy is {}/{} {} ".format(correct, pred_benign_test.shape[0]+pred_adv_test.shape[0], correct/(pred_benign_test.shape[0]+pred_adv_test.shape[0])))        

Accuracy is 6755/8293 0.8145423851440974 


In [119]:
# ### FOR ADV
# directory = 'data/twitter_data_new/adversary/twitter/target_next/jsma/'
# x_adv = np.load(directory+'/xadvclev_twitter.npy')
# y_pred_class_adv = np.empty(x_adv.shape[0])

# print('runing for adv')
# for i in range(x_adv.shape[0]):
#     if i%100 == 0:
#         print('i: ',i)
#     sample = x_adv[i]
#     exp = explainer.explain_instance(sample, predict_fn, num_features=5)
#     exp_l = exp.as_list(label=1)
#     intr_sample = preprocess_feature_vector(sample, exp_l)
#     try:
#         y_pred_class_adv[i] = intr_predict_class(intr_sample.reshape(1,-1))[0]
#     except:
#         print('failed for index ',i)
#         y_pred_class_adv[i] = 0

# np.save(out_directory+'/y_pred_class_adv.npy', y_pred_class_adv)

runing for adv
i:  0
i:  100
i:  200
i:  300
i:  400
i:  500
i:  600
i:  700
i:  800
i:  900
i:  1000
i:  1100
i:  1200
i:  1300
i:  1400
i:  1500
i:  1600
i:  1700
i:  1800
i:  1900
i:  2000
i:  2100
i:  2200
i:  2300
i:  2400
i:  2500
i:  2600
i:  2700
i:  2800
i:  2900
i:  3000
i:  3100
i:  3200
failed for index  3297
i:  3300
i:  3400
i:  3500
i:  3600
i:  3700
i:  3800
i:  3900
i:  4000
i:  4100
i:  4200
i:  4300
i:  4400
